In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_linnerud
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from mlxtend.plotting import plot_decision_regions
from copy import deepcopy
from pprint import pprint

In [2]:
df_titanic = pd.read_csv('/train.csv')
df_boston = pd.read_csv('/BostonHousing.csv')

In [3]:
df_titanic_factorized = pd.DataFrame(df_titanic)
df_titanic = df_titanic_factorized
for column in df_titanic_factorized.select_dtypes(include=['object']).columns:
    df_titanic_factorized[column], unique = pd.factorize(df_titanic_factorized[column])

In [ ]:
df_titanic_factorized.corr()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,1.000000,-0.005007,-0.035144,1.000000,-0.042939,0.036847,-0.057527,-0.001652,0.760875,0.012658,0.241918,-0.030323
Survived,-0.005007,1.000000,-0.338481,-0.005007,0.543351,-0.077221,-0.035322,0.081629,-0.047298,0.257307,0.270495,0.101849
Pclass,-0.035144,-0.338481,1.000000,-0.035144,-0.131900,-0.369226,0.083081,0.018443,-0.017489,-0.549500,-0.623554,0.050992
Name,1.000000,-0.005007,-0.035144,1.000000,-0.042939,0.036847,-0.057527,-0.001652,0.760875,0.012658,0.241918,-0.030323
Sex,-0.042939,0.543351,-0.131900,-0.042939,1.000000,-0.093254,0.114631,0.245489,-0.132709,0.182333,0.082104,0.111249
Age,0.036847,-0.077221,-0.369226,0.036847,-0.093254,1.000000,-0.308247,-0.189119,0.133553,0.096067,0.231448,0.002626
SibSp,-0.057527,-0.035322,0.083081,-0.057527,0.114631,-0.308247,1.000000,0.414838,-0.303229,0.159651,-0.058893,-0.058008
Parch,-0.001652,0.081629,0.018443,-0.001652,0.245489,-0.189119,0.414838,1.000000,-0.273002,0.216225,-0.003678,-0.076625
Ticket,0.760875,-0.047298,-0.017489,0.760875,-0.132709,0.133553,-0.303229,-0.273002,1.000000,-0.142578,0.212438,-0.020135
Fare,0.012658,0.257307,-0.549500,0.012658,0.182333,0.096067,0.159651,0.216225,-0.142578,1.000000,0.397105,0.058462


In [4]:
df_titanic = df_titanic_factorized.drop(columns = ['PassengerId', 'Name'] )

In [5]:
X = df_titanic.drop(columns=['Survived'])
y = df_titanic['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=3, stratify=y)

In [50]:

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        self.cnt = 0

    def gini_index(self, labels):
        _, counts = np.unique(labels, return_counts=True)
        probabilities = counts / counts.sum()
        gini = 1 - np.sum(probabilities ** 2)
        return gini

    def entropy(self, labels):
        _, counts = np.unique(labels, return_counts=True)
        probabilities = counts / counts.sum()
        ent = -np.sum(probabilities * np.log2(probabilities))
        return ent

    #Вторая функция
    def chosen_feature(self, X, y):
        ind = []
        for i in range(X.shape[1]):
            feature = X.iloc[:, i]
            _, gini = self.find_optimal_split(feature, y)
            ind.append(gini)
        min_index = ind.index(min(ind))
        column_name = X.columns[min_index]

        #print(f'Фича для разбиения {column_name}')
        return column_name

    #Третья функция
    def find_optimal_split(self, feature, labels):
        self.cnt += 1
        sorted_indices = np.argsort(feature)
        feature = np.array(feature)[sorted_indices]
        labels = np.array(labels)[sorted_indices]

        min_gini = float("inf") #представление положительной бесконечности
        optimal_threshold = None

        for i in range(1, len(feature)):
            threshold = (feature[i - 1] + feature[i]) / 2

            left_labels = labels[:i]
            right_labels = labels[i:]

            gini_left = self.gini_index(left_labels)
            gini_right = self.gini_index(right_labels)

            gini_total = (len(left_labels) * gini_left + len(right_labels) * gini_right) / len(labels)

            if gini_total < min_gini:
                min_gini = gini_total
                optimal_threshold = threshold

        #print(f'Оптимальные {self.cnt}: {optimal_threshold}, {min_gini}')

        return optimal_threshold, min_gini

    def split_data(self, feature, labels, optimal_threshold):

        left_data = []
        left_labels = []
        right_data = []
        right_labels = []

        for i in range(len(feature)):
            if feature[i] <= optimal_threshold:
                left_data.append(feature[i])
                left_labels.append(labels[i])
            else:
                right_data.append(feature[i])
                right_labels.append(labels[i])

        return (left_data, left_labels), (right_data, right_labels)

    #Запускается 1 после метода fit
    def build_tree(self, X, y, depth=0):
        # Базовый случай: если глубина превышена или все метки одинаковы
        if len(set(y)) == 1 or (self.max_depth and depth >= self.max_depth):
            return {"label": y.iloc[0]}

        # Выбор лучшего признака
        best_feature = self.chosen_feature(X, y)
        feature_index = X.columns.get_loc(best_feature)

        # Оптимальный порог для разделения
        threshold, _ = self.find_optimal_split(X.iloc[:, feature_index].values, y)

        # Разделение данных
        left_indices = X.iloc[:, feature_index].values <= threshold
        right_indices = X.iloc[:, feature_index].values >= threshold  #right_indices = ~left_indices другая форма записи


        left_tree = self.build_tree(X[left_indices], y[left_indices], depth + 1)
        right_tree = self.build_tree(X[right_indices], y[right_indices], depth + 1)

        tree = {
            "feature": best_feature,
            "threshold": threshold,
            "left": left_tree,
            "right": right_tree,
        }

        print(f'Фича {tree["feature"]}')
        print(f'Порог {tree["threshold"]}')
        print(f'Левая часть {tree["left"]}')
        print(f'Правая часть {tree["right"]}')

        return tree

    def fit(self, X, y):
        self.tree = self.build_tree(X, y)

    def predict_sample(self, sample, tree):
        if "label" in tree:
            return tree["label"]

        feature = tree["feature"]
        threshold = tree["threshold"]
        if sample[feature] <= threshold:
            return self.predict_sample(sample, tree["left"])
        else:
            return self.predict_sample(sample, tree["right"])

    def predict(self, X, y):
        predictions = np.array([self.predict_sample(sample, self.tree)
                                for _, sample in X.iterrows()])

        acc = accuracy_score(y, predictions)
        #print(f'Точность модели: {acc}')
        return predictions



In [31]:
tree = DecisionTree(max_depth=3)

In [32]:
tree.fit(X_train, y_train)

Оптимальные 1: 2.0, 0.415606288979131
Оптимальные 2: 0.5, 0.3447544862076387
Оптимальные 3: nan, 0.4947729200441328
Оптимальные 4: 2.0, 0.46528072914898677
Оптимальные 5: 0.0, 0.4623345188319751


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)


Оптимальные 6: 564.5, 0.46809911577384283
Оптимальные 7: 10.5, 0.4344112901326172
Оптимальные 8: -0.5, 0.43084301222765
Оптимальные 9: 0.0, 0.4610225156556375
Фича для разбиения Sex
Оптимальные 10: 0.5, 0.3447544862076387
Оптимальные 11: 1.0, 0.2981016053141172
Оптимальные 12: 0.0, 0.31641056100551807
Оптимальные 13: 33.5, 0.24152968069618927
Оптимальные 14: 0.0, 0.3157305467437955


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)


Оптимальные 15: 1.0, 0.31176861412003176
Оптимальные 16: 648.5, 0.31781797125144773
Оптимальные 17: 26.26875, 0.30540016221398275
Оптимальные 18: 34.5, 0.294772019948422
Оптимальные 19: 2.0, 0.3162351104318314
Фича для разбиения Age
Оптимальные 20: 6.5, 0.2970245770703824
Оптимальные 21: 2.5, 0.3
Оптимальные 22: 0.0, 0.31111111111111106
Оптимальные 23: 1.0, 0.3428571428571429
Оптимальные 24: 2.0, 0.08571428571428573
Оптимальные 25: 2.0, 0.375
Оптимальные 26: 67.0, 0.15
Оптимальные 27: 20.825, 0.3
Оптимальные 28: -1.0, 0.23516483516483505
Оптимальные 29: 0.0, 0.31111111111111106
Фича для разбиения SibSp
Оптимальные 30: 2.0, 0.08571428571428573
Оптимальные 31: 3.0, 0.14285714285714285
Оптимальные 32: 0.0, 0.14285714285714285
Оптимальные 33: 2.5, 0.14285714285714285
Оптимальные 34: 4.0, 0.19047619047619047
Оптимальные 35: 2.0, 0.0
Оптимальные 36: 20.0, 0.21428571428571427
Оптимальные 37: 31.331249999999997, 0.21428571428571427
Оптимальные 38: -1.0, 0.14285714285714285
Оптимальные 39: 0.0,

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)


Оптимальные 88: 129.5, 0.11310822510822516
Оптимальные 89: 0.0, 0.11021184934228417
Фича для разбиения Age
Оптимальные 90: 2.0, 0.10039324543141337
Оптимальные 91: 1.5, 0.0
Оптимальные 92: 1.0, 0.0
Оптимальные 93: 2.0, 0.0
Оптимальные 94: 1.0, 0.0
Оптимальные 95: 1.5, 0.0
Оптимальные 96: 344.5, 0.0
Оптимальные 97: 88.775, 0.0
Оптимальные 98: 26.0, 0.0
Оптимальные 99: 0.0, 0.0
Фича для разбиения Pclass
Оптимальные 100: 1.5, 0.0
Фича Pclass
Порог 1.5
Левая часть {'label': np.int64(0)}
Правая часть {'label': np.int64(1)}
Оптимальные 101: 2.0, 0.11712432240291494
Оптимальные 102: 1.0, 0.11914537075827401
Оптимальные 103: 2.0, 0.10647783897193806
Оптимальные 104: 1.0, 0.11909128260629934
Оптимальные 105: 2.0, 0.11940416296734743
Оптимальные 106: 312.5, 0.11816545973228007
Оптимальные 107: 26.0, 0.11673313065108368
Оптимальные 108: 54.0, 0.11977651275563986
Оптимальные 109: 0.0, 0.11681471092410786
Фича для разбиения Age
Оптимальные 110: 2.0, 0.10647783897193806
Фича Age
Порог 2.0
Левая част

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)


Оптимальные 118: -1.0, 0.44340172682150264
Оптимальные 119: 1.0, 0.44860541115671065
Фича для разбиения Pclass
Оптимальные 120: 3.0, 0.3784606866002214
Оптимальные 121: 3.0, 0.3784606866002214
Оптимальные 122: 1.0, 0.44830613284186543
Оптимальные 123: 34.0, 0.41523724880264656
Оптимальные 124: 2.0, 0.41324373301117484
Оптимальные 125: 1.0, 0.4429891813612744
Оптимальные 126: 227.5, 0.4486039270749054
Оптимальные 127: 27.825, 0.4421361531109858
Оптимальные 128: -1.0, 0.44340172682150264
Оптимальные 129: 1.0, 0.44860541115671065
Фича для разбиения Pclass
Оптимальные 130: 3.0, 0.3784606866002214
Фича Pclass
Порог 3.0
Левая часть {'label': np.int64(1)}
Правая часть {'label': np.int64(1)}
Оптимальные 131: 3.0, 0.4786062378167642
Оптимальные 132: 1.0, 0.4786062378167642
Оптимальные 133: nan, 0.441358024691358
Оптимальные 134: 2.5, 0.4610313662945243
Оптимальные 135: 1.0, 0.4624218774785064
Оптимальные 136: 164.0, 0.4870380044403813
Оптимальные 137: 23.35, 0.41097367073597973
Оптимальные 138:

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)


In [30]:
tree.predict(X_test, y_test) #глубина 3

Точность модели: 0.8379888268156425


array([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 1])

In [33]:
tree.predict(X_test, y_test) #глубина 4

Точность модели: 0.7877094972067039


array([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 1])

**RandomForest**

In [46]:
import random

class RandomForest:
    def __init__(self, n_trees=10, max_depth=None, sample_size=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.sample_size = sample_size
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            sample_indices = random.sample(range(len(X)), self.sample_size or len(X))
            sample_X = X.iloc[sample_indices]
            sample_y = y.iloc[sample_indices]

            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(sample_X, sample_y)
            self.trees.append(tree)

    def predict(self, X, y):
        tree_predictions = [tree.predict(X, y) for tree in self.trees]
        # Усреднение предсказаний деревьев (например, для классификации)
        pred = np.array([max(set(preds), key=preds.count) for preds in zip(*tree_predictions)])
        acc = accuracy_score(y, pred)
        print(f'Точность модели: {acc}')
        return pred


In [54]:
X_trainf, X_testf, y_trainf, y_testf = train_test_split(X, y, train_size = 0.8, random_state=3, stratify=y)

In [51]:
ftree = RandomForest(max_depth = 3)

In [55]:
ftree.fit(X_trainf, y_trainf)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)


Фича Age
Порог 5.5
Левая часть {'label': np.int64(1)}
Правая часть {'label': np.int64(0)}
Фича Age
Порог 27.0
Левая часть {'label': np.int64(1)}
Правая часть {'label': np.int64(1)}
Фича Cabin
Порог 34.5
Левая часть {'feature': 'Age', 'threshold': np.float64(5.5), 'left': {'label': np.int64(1)}, 'right': {'label': np.int64(0)}}
Правая часть {'feature': 'Age', 'threshold': np.float64(27.0), 'left': {'label': np.int64(1)}, 'right': {'label': np.int64(1)}}


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)


Фича Cabin
Порог 34.5
Левая часть {'label': np.int64(0)}
Правая часть {'label': np.int64(1)}


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)


...

Фича Sex
Порог 0.0
Левая часть {'label': np.int64(0)}
Правая часть {'label': np.int64(0)}
Фича Sex
Порог 0.0
Левая часть {'feature': 'Age', 'threshold': np.float64(6.5), 'left': {'label': np.int64(1)}, 'right': {'label': np.int64(0)}}
Правая часть {'feature': 'Sex', 'threshold': np.float64(0.0), 'left': {'label': np.int64(0)}, 'right': {'label': np.int64(0)}}
Фича Sex
Порог 0.0
Левая часть {'feature': 'Age', 'threshold': np.float64(6.5), 'left': {'feature': 'SibSp', 'threshold': np.float64(2.0), 'left': {'label': np.int64(1)}, 'right': {'label': np.int64(1)}}, 'right': {'feature': 'Cabin', 'threshold': np.float64(33.0), 'left': {'label': np.int64(0)}, 'right': {'label': np.int64(1)}}}
Правая часть {'feature': 'Sex', 'threshold': np.float64(0.0), 'left': {'feature': 'Age', 'threshold': np.float64(6.5), 'left': {'label': np.int64(1)}, 'right': {'label': np.int64(0)}}, 'right': {'feature': 'Sex', 'threshold': np.float64(0.0), 'left': {'label': np.int64(0)}, 'right': {'label': np.int64(0)}

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)


Фича Sex
Порог 1.0
Левая часть {'label': np.int64(1)}
Правая часть {'label': np.int64(1)}
Фича Pclass
Порог 3.0
Левая часть {'label': np.int64(1)}
Правая часть {'label': np.int64(0)}
Фича Sex
Порог 1.0
Левая часть {'feature': 'Sex', 'threshold': np.float64(1.0), 'left': {'label': np.int64(1)}, 'right': {'label': np.int64(1)}}
Правая часть {'feature': 'Pclass', 'threshold': np.float64(3.0), 'left': {'label': np.int64(1)}, 'right': {'label': np.int64(0)}}


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be order

Фича Pclass
Порог 3.0
Левая часть {'label': np.int64(1)}
Правая часть {'label': np.int64(0)}
Фича Fare
Порог 23.35
Левая часть {'label': np.int64(0)}
Правая часть {'label': np.int64(0)}
Фича Pclass
Порог 3.0
Левая часть {'feature': 'Pclass', 'threshold': np.float64(3.0), 'left': {'label': np.int64(1)}, 'right': {'label': np.int64(0)}}
Правая часть {'feature': 'Fare', 'threshold': np.float64(23.35), 'left': {'label': np.int64(0)}, 'right': {'label': np.int64(0)}}
Фича Sex
Порог 1.0
Левая часть {'feature': 'Sex', 'threshold': np.float64(1.0), 'left': {'feature': 'Sex', 'threshold': np.float64(1.0), 'left': {'label': np.int64(1)}, 'right': {'label': np.int64(1)}}, 'right': {'feature': 'Pclass', 'threshold': np.float64(3.0), 'left': {'label': np.int64(1)}, 'right': {'label': np.int64(0)}}}
Правая часть {'feature': 'Pclass', 'threshold': np.float64(3.0), 'left': {'feature': 'Pclass', 'threshold': np.float64(3.0), 'left': {'label': np.int64(1)}, 'right': {'label': np.int64(0)}}, 'right': {'f

In [56]:
ftree.predict(X_testf, y_testf)

Точность модели: 0.5977653631284916


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])